# Unsupervised Valence Assignment for Sentiment Analysis

In [6]:
import urllib
import numpy as np
import json
import re
import sys
import time
import sqlite3 as sl
import pandas as pd
from bs4 import BeautifulSoup
import scipy.sparse

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize
from nltk.corpus import stopwords

In [7]:
first_known_file = 1
last_known_file = 67790
#last_known_file = 1000
max_file_size = 0

In [42]:
'''
for web_idx in range(first_known_file, last_known_file):
    url = f"https://gutenberg.org/files/{web_idx}/"
    try:
        file = urllib.request.urlopen(url)
    except urllib.error.HTTPError as e: # If the page is forbiden, then move on
        continue
    except urllib.error.URLError as e: # if we loose connection, try again
        time.sleep(4)
        web_idx -= 4

    # Looking for every .txt.utf-8 file given the above url
    online_file = list()
    for line in file:
        decoded_line = None
        try:
            decoded_line = line.decode("utf-8")
        except UnicodeDecodeError as e:
            continue

        online_file.append(decoded_line)



    # For each .txt.utf-8 file we need to create a new url to get to the file
    res = [string for string in online_file if '.txt">' in string]
    txt_urls = list()
    txt_file_name = None
    for line in res:
        result = re.search('<a href="(.*).txt">', line)
        if result != None:
            num_groups = len(result.groups())

            if num_groups > 0:
                txt_file_name = f"{result.group(1)}.txt"
                txt_urls.append(f"{url}{txt_file_name}")

    # for each .txt.utf-8 url file, read in the file and get all of the text
    txt_file_content = list()
    line_idx = 0
    for url_idx in range(len(txt_urls)):
        url = txt_urls[url_idx]

        if url == None or url == '':
            continue

        file = urllib.request.urlopen(url)
        line_by_line = list()
        for line in file:
            try:
                decoded_line = line.decode("utf-8")
            except UnicodeDecodeError as e:
                continue
            line_by_line.append(decoded_line)  
        txt_file_content.append(line_by_line)

        size_of_file = sys.getsizeof(line_by_line)
        if size_of_file > max_file_size:
            max_file_size = size_of_file
        print(f"{url}: {size_of_file} / {max_file_size}")
    
    # Write file to local disk
    #txt_file_content_np = np.asarray(txt_file_content, str)
    #txt_file_content_np = txt_file_content_np.flatten()
    if len(txt_file_content) >= 1 and len(txt_file_content[0]) >= 1:
        with open(f'raw/{txt_file_name}','w') as f:
            if f.tell() < 1024*1024*1024:
                #pickle.dump(txt_file_content_np, f)
                f.write(json.dumps(txt_file_content))
                #for line in txt_file_content:
                    #f.write('%s\n' % line)
'''

https://gutenberg.org/files/67770/67770-0.txt: 33048 / 33048
https://gutenberg.org/files/67771/67771-0.txt: 33048 / 33048
https://gutenberg.org/files/67772/67772-0.txt: 11288 / 33048
https://gutenberg.org/files/67773/67773-0.txt: 95864 / 95864
https://gutenberg.org/files/67774/67774-0.txt: 85176 / 95864
https://gutenberg.org/files/67775/67775-0.txt: 37208 / 95864
https://gutenberg.org/files/67776/67776-0.txt: 29336 / 95864
https://gutenberg.org/files/67777/67777-0.txt: 29336 / 95864
https://gutenberg.org/files/67778/67778-0.txt: 95864 / 95864
https://gutenberg.org/files/67779/67779-0.txt: 53080 / 95864
https://gutenberg.org/files/67780/67780-0.txt: 59736 / 95864


In [33]:
#with open('raw/10.pkl','rb') as f:
#    x = pickle.load(f)
#    print(x[0][125])

1:8 And God called the firmament Heaven. And the evening and the



## Example document


## ---------------


The Project Gutenberg EBook of Madame de Ferneuse, by Daniel Lesueur

This eBook is for the use of anyone anywhere in the United States and most other parts of the world at no cost and with almost no restrictions whatsoever. You may copy it, give it away or re-use it under the terms of the Project Gutenberg License included with this eBook or online at www.gutenberg.org. If you are not located in the United States, you'll have to check the laws of the country where you are located before using this ebook.

Title: Madame de Ferneuse

Author: Daniel Lesueur

Release Date: March 21, 2016 [EBook #51515]

Language: French

Character set encoding: UTF-8

*** START OF THIS PROJECT GUTENBERG EBOOK MADAME DE FERNEUSE ***

*** END OF THIS PROJECT GUTENBERG EBOOK MADAME DE FERNEUSE ***

***** This file should be named 51515-0.txt or 51515-0.zip ***** This and all associated files of various formats will be found in: http://www.gutenberg.org/5/1/5/1/51515/


## ------------------

The Project Gutenberg EBook of The Meaning of Infancy, by John Fiske

This eBook is for the use of anyone anywhere at no cost and with almost no restrictions whatsoever. You may copy it, give it away or re-use it under the terms of the Project Gutenberg License included with this eBook or online at www.gutenberg.org

Title: The Meaning of Infancy

Author: John Fiske

Release Date: May 15, 2004 [EBook #12359]

Language: English

Character set encoding: ASCII

*** START OF THIS PROJECT GUTENBERG EBOOK THE MEANING OF INFANCY ***

*** END OF THIS PROJECT GUTENBERG EBOOK THE MEANING OF INFANCY ***

## -------------------

The Project Gutenberg eBook of Traumerei, by Charles Beaumont

This eBook is for the use of anyone anywhere in the United States and most other parts of the world at no cost and with almost no restrictions whatsoever. You may copy it, give it away or re-use it under the terms of the Project Gutenberg License included with this eBook or online at www.gutenberg.org. If you are not located in the United States, you will have to check the laws of the country where you are located before using this eBook.

Title: Traumerei

Author: Charles Beaumont

Release Date: February 7, 2022 [eBook #67341]

Language: English

Produced by: Greg Weeks, Mary Meehan and the Online Distributed Proofreading Team at http://www.pgdp.net

*** START OF THE PROJECT GUTENBERG EBOOK TRAUMEREI ***

*** END OF THE PROJECT GUTENBERG EBOOK TRAUMEREI ***

## -------------------


### New Design
Right now, we are downloading all of the files, saving them as raw array format as a json file.

Alternative approach as relational database
We can generate the hypterlink by walking a list from 0 to last_known_file. We would need to check what files they have. Then, save only the heading information in each document, and remember where the starting index of the literature as well as the ending index.  

Format tipes:
<ul>
    <li>.txt</li>
    <li>-0.txt</li>
    <li>-8.txt</li>
    <li>-h/</li>
    <li>-0.zip</li>
    <li>-8.zip</li>
    <li>-h.zip</li>
</ul>

from there, we can read the txt file, assuming it can be decoded using utf-8. The reoccuring headings can allow us to save data entries into a relational database.
Document: (some of these fields may not be nan)
<ul>
    <li>Title:</li>
    <li>Author:</li>
    <li>URL:</li>
    <li>Release Date:</li>
    <li>Edition:</li>
    <li>Language:</li>
    <li>Produced by:</li>
    <li>Start_Of_doc_indx:</li>
    <li>End_of_doc_indx:</li>
    <li>(or, if we have the memory, the entire document, but for now assuming we are reading from a URL or File Stream)</li>
</ul>



Python and SQLite3

Medium post by Christopher Tao: https://towardsdatascience.com/do-you-know-python-has-a-built-in-database-d553989c87bd

Tutorialspoint python SQLlite: https://towardsdatascience.com/do-you-know-python-has-a-built-in-database-d553989c87bd

# SQLite database  

example code to work with sqlite3

In [8]:
con = sl.connect('my-test.db')

with con:
    con.execute("""
        CREATE TABLE USER (
            id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
            name TEXT,
            age INTEGER
        );
    """)

sql = 'INSERT INTO USER (id, name, age) values(?, ?, ?)'
data = [
    (1, 'Alice', 21),
    (2, 'Bob', 22),
    (3, 'Chris', 23)
]

with con:
    con.executemany(sql, data)

In [12]:
with con:
    data = con.execute("SELECT * FROM USER WHERE age <= 22")
    for row in data:
        print(row)

(1, 'Alice', 21)
(2, 'Bob', 22)


Integrate with Pandas

In [13]:
df_skill = pd.DataFrame({
    'user_id': [1,1,2,2,3,3,3],
    'skill': ['Network Security', 'Algorithm Development', 'Network Security', 'Java', 'Python', 'Data Science', 'Machine Learning']
})

In [14]:
df_skill.to_sql('SKILL', con)

7

In [16]:
df = pd.read_sql('''
    SELECT s.user_id, u.name, u.age, s.skill 
    FROM USER u LEFT JOIN SKILL s ON u.id = s.user_id
''', con)
df

,user_id,name,age,skill
0,1,Alice,21,Algorithm Development
1,1,Alice,21,Network Security
2,2,Bob,22,Java
3,2,Bob,22,Network Security
4,3,Chris,23,Data Science
5,3,Chris,23,Machine Learning
6,3,Chris,23,Python


In [17]:
df.to_sql('USER_SKILL', con)

7

Implementing Refactor

In [16]:
db_con = sl.connect('gutenberg.db')

with db_con:
    db_con.execute("""
        CREATE TABLE LITERATURE (
            id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
            title TEXT,
            author TEXT,
            root_url TEXT,
            release_date TEXT,
            edition INTEGER,
            language TEXT,
            produced_by TEXT,
            translator TEXT,
            url_txt_0 TEXT,
            url_txt_8 TEXT,
            url_zip_0 TEXT,
            url_zip_8 TEXT,
            url_zip_h TEXT,
            url_h TEXT,
            url_txt TEXT
        );
    """)

In [4]:
#sql = 'INSERT INTO USER (id, name, age) values(?, ?, ?)'
#data = [
#    (1, 'Alice', 21),
#    (2, 'Bob', 22),
#    (3, 'Chris', 23)
#]
#
#with con:
#    con.executemany(sql, data)

In [5]:
#db_con = sl.connect('gutenberg.db')
#
#with con:
#    con.execute("""DROP table LITERATURE;""")

In [17]:
TXT_0_FILE = '-0.txt'
TXT_8_FILE = '-8.txt'
ZIP_0_FILE = '-0.zip'
ZIP_8_FILE = '-8.zip'
ZIP_H_FILE = '-h.zip'
H_FILE = '-h/'
TXT_FILE = '.txt'

In [ ]:
for web_idx in range(1, last_known_file):
    # variables to populate in database
    id = web_idx
    title = "None"
    author = "None"
    root_url = f"https://gutenberg.org/files/{web_idx}/"
    release_date = "None"
    edition = 0
    language = "None"
    produced_by = "None"
    translator = "None"
    url_txt_0 = "None"
    url_txt_8 = "None"
    url_zip_0 = "None"
    url_zip_8 = "None"
    url_zip_h = "None"
    url_h = "None"
    url_txt = "None"

    # Connect to project gutenberg
    try:
        file = urllib.request.urlopen(root_url)
    except urllib.error.HTTPError as e: # If the page is forbiden, then move on
        continue
    except urllib.error.URLError as e: # if we loose connection, try again
        time.sleep(4)
        web_idx -= 4

    # Strip web string from php syntax
    online_file = list()
    for line in file:
        decoded_line = None
        try:
            decoded_line = line.decode("utf-8")
        except UnicodeDecodeError as e:
            continue
        readable_line = ''.join(BeautifulSoup(decoded_line).findAll(text=True))
        online_file.append(readable_line)
    fullstring = ''.join(online_file)

    # Parese web file for txt files  
    if f'{web_idx}{TXT_0_FILE}' in fullstring:
        url_txt_0 = f"https://gutenberg.org/files/{web_idx}/{web_idx}{TXT_0_FILE}"
    if f'{web_idx}{TXT_8_FILE}' in fullstring:
        url_txt_8 = f"https://gutenberg.org/files/{web_idx}/{web_idx}{TXT_8_FILE}"
    if f'{web_idx}{ZIP_0_FILE}' in fullstring:
        url_zip_0 = f"https://gutenberg.org/files/{web_idx}/{web_idx}{ZIP_0_FILE}"
    if f'{web_idx}{ZIP_8_FILE}' in fullstring:
        url_zip_8 = f"https://gutenberg.org/files/{web_idx}/{web_idx}{ZIP_8_FILE}"
    if f'{web_idx}{ZIP_H_FILE}' in fullstring:
        url_zip_h = f"https://gutenberg.org/files/{web_idx}/{web_idx}{ZIP_H_FILE}"
    if f'{web_idx}{H_FILE}' in fullstring:
        url_h = f"https://gutenberg.org/files/{web_idx}/{web_idx}{H_FILE}"
    if f'{web_idx}{TXT_FILE}' in fullstring:
        url_txt = f"https://gutenberg.org/files/{web_idx}/{web_idx}{TXT_FILE}"

    # use one of the txt files to gather remanining variables
    file_url = None
    if url_txt_0:
        file_url = url_txt_0
    elif url_txt_8:
        file_url = url_txt_8
    elif url_txt:
        file_url = url_txt

    if file_url == "None":
        continue

    # Connect to txt file
    file = None
    try:
        file = urllib.request.urlopen(file_url)
    except urllib.error.HTTPError as e: # If the page is forbiden, then move on
        continue
    except urllib.error.URLError as e: # if we loose connection, try again
        time.sleep(4)
        web_idx -= 4

    # Strip txt file from php syntax
    txt_file = list()
    for line in file:
        decoded_line = None
        try:
            decoded_line = line.decode("utf-8")
        except UnicodeDecodeError as e:
            continue
        #readable_line = ''.join(BeautifulSoup(decoded_line).findAll(text=True))
        txt_file.append(decoded_line)
    full_txt = ''.join(txt_file)

    #print(full_txt)

    # Parse text file for
    title_results = re.search('Title: (.*)', full_txt)
    author_results = re.search('Author: (.*)', full_txt)
    release_data_results = re.search('Release Date: (.*) \[', full_txt)
    edition_results = re.search('Edition: (.*)', full_txt)
    language_results = re.search('Language: (.*)', full_txt)
    produced_results = re.search('Produced by: (.*)', full_txt)
    translator_results = re.search('Translator: (.*)', full_txt)

    if title_results:
        title = str(title_results.group(1)).strip()
    if author_results:
        author = str(author_results.group(1)).strip()
    if release_data_results:
        release_date = str(release_data_results.group(1)).strip()
    if edition_results:
        edition_str = str(edition_results.group(1)).strip()
        if edition_str.isnumeric():
            edition = int(edition_str)
    if language_results:
        language = str(language_results.group(1)).strip()
    if produced_results:
        produced_by = str(produced_results.group(1)).strip()
    if translator_results:
        translator = str(translator_results.group(1)).strip()


    #print(f"id: {type(id)}")
    #print(f"title: {type(title)}")
    #print(f"author: {type(author)}")
    #print(f"root_url: {type(root_url)}")
    #print(f"release_date: {type(release_date)}")
    #print(f"edition: {type(edition)}")
    #print(f"language: {type(language)}")
    #print(f"produced_by: {type(produced_by)}")
    #print(f"url_txt_0: {type(url_txt_0)}")
    #print(f"url_txt_8: {type(url_txt_8)}")
    #print(f"url_zip_0: {type(url_zip_0)}")
    #print(f"url_zip_8: {type(url_zip_8)}")
    #print(f"url_zip_h: {type(url_zip_h)}")
    #print(f"url_h: {type(url_h)}")
    #print(f"url_txt: {type(url_txt)}")
    
    sql = 'INSERT INTO LITERATURE (id, title, author, root_url, release_date, edition, language, produced_by, translator, url_txt_0, url_txt_8, url_zip_0, url_zip_8, url_zip_h, url_h, url_txt) values(?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)'
    data = [(id, title, author, root_url, release_date, edition, language, produced_by, translator, url_txt_0, url_txt_8, url_zip_0, url_zip_8, url_zip_h, url_h, url_txt)]

    with db_con:
        db_con.executemany(sql, data)

In [25]:
db_con = sl.connect('gutenberg.db')

with db_con:
    db_con.execute("""DROP table LITERATURE;""")

    db_con.execute("""
        CREATE TABLE LITERATURE (
            id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
            title TEXT,
            author TEXT,
            root_url TEXT,
            release_date TEXT,
            edition INTEGER,
            language TEXT,
            produced_by TEXT,
            translator TEXT,
            url_txt_0 TEXT,
            url_txt_8 TEXT,
            url_zip_0 TEXT,
            url_zip_8 TEXT,
            url_zip_h TEXT,
            url_h TEXT,
            url_txt TEXT
        );
    """)

## Encoding Representations


- One-Hot encoding
    - Each word is represented as a single dimension

![image.png](pic/fig_1_1.png)

- Vector Space Model (VSM)
    - Refered to as the "Semantic Space"
    - Representation of the object is "Distributed Representation"
        - object can be words, documents, sentences, concepts, or entities 
        
![image.png](pic/fig_1_2.png)

Word Embeddings


Count-based models

    - term-document (row: words, col: documents) denoting the frequency of a specific word in a doc. Meant for document centirc work (document retrieval, classification, or similar document)
    - Word-context. (representing words) <---
    - pair-pattern (row: pairs of words, col: pattern occurance)



Positive Pointwise Mutual Infromation model

    - better than raw frequencies.
    - any negative values are clampped to 0
    - checks if w_1 and w_2 co-occur more than they occur independelty
    - negative values means that two words co-occurens is less than likely to happen than by chance

$ PMI(w_1, w_2) = log_2 \dfrac{P(w_1,w_2)}{P(w_1)P(w_2)}$



Cosine Similarity

    - dot product(v, w) where v and w are the embeddings of the word v and w.
    - or the cos(v,w) = (v dot w)/|v||w|


tf-idf model

    - term frequency - the frequency of a word within a document
    - inverse document frequency - the number of documents the words was found in
    - collection frequency - the total number of times a words was found in all documents
    - can look at the window as the "document"


The simplest embeddings we can do is creating a matrix that is n by n, where n is the number of unique words in the corpus. Then use the PPMI metric to calculate the probability of the words occuring at all in the corpus.


Another representation can be the word count of pairs of words, but only incrementing if the other word is +/- n words away from the target word. Essentially a sliding window, where the midpoint is the target word, and the outside words are the occurances of that word bing visable. This creates the impression of word relations between other words, with a small window allowing for specific connections, and larger windows give general connections.


Dimensionality reduction can help with the previous matrix to reduce the sparsity of the matrix. (SVD and PCA). Note that the new columns are now not naturally interpretable, whereas the oriignal embedingnig can be interpretable.


Recomended that we only keep track of the first 50,000 most frequent words, removing stop words, to reduce the sparsity of the embedding. Having more would just cause noise.


Random Indexing

    - "RI first generates unique and random lowdimensional representations for words, called index vectors. Then, in an incremental process while scanning through the text corpus, each time a word c is observed in the context of a target word w, the index vector for w is updated by adding to it the index vector of c. 
    - The resulting representations are an approximation of the co-occurrence matrix, but with a significantly lower dimensionality, and without the need for costly SVD computations. Moreover, RI representations can be update after the initial training, and once new data is at disposal."


Predictive Models

    - Using neural networks to predict the embedding space of the words.
    - Word2Vec
    - GloVe

Character Embeddings

    - out of vocaulary (OOV)
    - solution to randomly assign a value, but can hinder our understanding/accuracy
    - using morphological segmenter (memoryless --> memory and less)
    - broken into constituent subwords (groups of characters that are not neccessarily semantically meaningful)
    - FastText
    - Byte pair encoding, and WordPiece 
    - WordNet

Knowledge-enhanced word embeddings
    - retrofitting models


Evaluation of word embeddings
    - Intrinsic (generic evaluation)
    - Extrinsic (specific application use evaluation)




In [8]:
# Need an abstract class that holds the "embeddings" method
# All we need to know is that the model works with an arbitrary embedding
# which at the end of the day it is an n by n matrix.

# This may be just a wrapper class for sklearn feature extraction 

# Sparse matrix representation by having 3 parallel lists
# row holds the row index
# col holds the col index
# data holds the value at (row, col)
class Embedding():
    def __init__(self):
        self.row = list()
        self.col = list()
        self.data = list()



# we will be working with sparse matrixies, and thus we can use scipy's sparce matrix package
# example of using their scipy.sparse.csr_array (Compressed Sparse Row array)
# row = np.array([0, 0, 1, 2, 2, 2])
# col = np.array([0, 2, 2, 0, 1, 2])
# data = np.array([1, 2, 3, 4, 5, 6])
# csr_array((data, (row, col)), shape=(3, 3)).toarray()
#    array([[1, 0, 2],
#           [0, 0, 3],
#           [4, 5, 6]])
# 
# The tuple to represent row and col are seperated in the row and col parallel vectors.
# Then a third parallel vector represents the data that we are interested in
# 
# Addtionlay, if there is a duplicate row/col entries, these are summed together
# row = np.array([0, 1, 2, 0])
# col = np.array([0, 1, 1, 0])
# data = np.array([1, 2, 4, 8])
# csr_array((data, (row, col)), shape=(3, 3)).toarray()
#   array([[9, 0, 0],
#          [0, 2, 0],
#          [0, 4, 0]])

Understanding how to use sklearn feature extractions

In [9]:
def unique(list1):
    unique_word_list = []
    for x in list1:
        sentence_list = x.split()
        for y in sentence_list:
            lower = y.lower()
            if lower not in unique_word_list:
                unique_word_list.append(lower)
    return unique_word_list



corpus = [
    'This is the first document',
    'This is the second second document',
    'And the third one',
    'Is this the first document',
]

vocab_list = unique(corpus)
vocab_dict = {word: idx for idx, word in enumerate(vocab_list)}
vocab_dict

{'this': 0,
 'is': 1,
 'the': 2,
 'first': 3,
 'document': 4,
 'second': 5,
 'and': 6,
 'third': 7,
 'one': 8}

In [10]:
vectorizer = CountVectorizer(vocabulary=vocab_dict)
X = vectorizer.fit_transform(corpus)
X.toarray()

array([[1, 1, 1, 1, 1, 0, 0, 0, 0],
       [1, 1, 1, 0, 1, 2, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 1, 1, 1],
       [1, 1, 1, 1, 1, 0, 0, 0, 0]], dtype=int64)

In [11]:
from os import listdir
from os.path import isfile, join

def GetListOfFileNames(dir_path):
    return [join(dir_path, f) for f in listdir(dir_path) if isfile(join(dir_path, f))]


Getting File paths to raw data

In [14]:
DIR_OF_TEXT = "C:/Users/Jake Klinkert/Documents/School/Spring 2022/CS8390 NLP/final project/code/data/text"
DIR_OF_COUNTS = "C:/Users/Jake Klinkert/Documents/School/Spring 2022/CS8390 NLP/final project/code/data/counts"
DIR_OF_TOKENS = "C:/Users/Jake Klinkert/Documents/School/Spring 2022/CS8390 NLP/final project/code/data/tokens"


file_path_texts = GetListOfFileNames(DIR_OF_TEXT)
file_path_texts.pop(0)

file_path_counts = GetListOfFileNames(DIR_OF_COUNTS)
file_path_counts.pop(0)

file_path_tokens = GetListOfFileNames(DIR_OF_TOKENS)
file_path_tokens.pop(0)

'C:/Users/Jake Klinkert/Documents/School/Spring 2022/CS8390 NLP/final project/code/data/tokens\\.dummy'

Processing data into embedding space

In [15]:
vectorizer_spgc = CountVectorizer(  input="filename", 
                                    encoding='utf-8',
                                    strip_accents='ascii',
                                    lowercase=True,
                                    stop_words=stopwords.words('english'),
                                    ngram_range = (1, 1))

count_embeding = vectorizer_spgc.fit_transform(file_path_texts)

count_embeding is a document-term matrix, where each row is a document and each featrue is a token. We may want to transpose this so that the first index is the word, and the second index is the document.

From here, we want to perform vector arithmatic to determine how "similar" the words are in relation to a polar set of words. To do this we will need to:

    - Get the polar word vectors (hate/love)
    - Determine the vector between these by subtracting each other (love - hate)
    - SemAxis will then need to be normalized ([love-hate]/|[love-hate]|)
    - Project all terms vectors onto this SemAxis
    - Determine the distance of the projected term from the negative word
    - normalize distance such that it is from [0-1]

![image.png](pic/revised_occ.png)

In [23]:
polar_word_sets =[
    ["negative", "positive"],
    ["displeased", "pleased"],
    ["disapproving", "approving"],
    ["disliking", "liking"],
    ["fear", "hope"],
    ["distress", "joy"],
    ["shame", "pride"],
    ["reproach", "admiration"],
    ["hate", "love"],
    ["disgust", "interest"],
    ["fears-confirmed", "satisfaction"],
    ["disappointment", "relief"],
    ["resentment", "happy-for"],
    ["pity", "gloating"]
]

polar_word_index =[
    [-1, -1],
    [-1, -1],
    [-1, -1],
    [-1, -1],
    [-1, -1],
    [-1, -1],
    [-1, -1],
    [-1, -1],
    [-1, -1],
    [-1, -1],
    [-1, -1],
    [-1, -1],
    [-1, -1],
    [-1, -1]
]

In [24]:
feature_list = vectorizer_spgc.get_feature_names_out()

In [25]:
for word_set_idx in range(len(polar_word_sets)):
    for word_idx in range(len(polar_word_sets[word_set_idx])):
        for (index, item) in enumerate(feature_list):
            if item == polar_word_sets[word_set_idx][word_idx]:
                print(f"{item} at index {index}")
                polar_word_index[word_set_idx][word_idx] = index
                break

print(polar_word_index)

negative at index 7369506
positive at index 8030569
displeased at index 3928681
pleased at index 7966067
disapproving at index 3912933
approving at index 1964729
disliking at index 3924900
liking at index 6666615
fear at index 4440214
hope at index 5743119
distress at index 3937670
joy at index 6103383
shame at index 8811288
pride at index 8091041
reproach at index 8387214
admiration at index 1530527
hate at index 5565653
love at index 6735188
disgust at index 3922027
interest at index 5963450
satisfaction at index 8633246
disappointment at index 3912733
relief at index 8360014
resentment at index 8393765
pity at index 7950937
gloating at index 5220677
[[7369506, 8030569], [3928681, 7966067], [3912933, 1964729], [3924900, 6666615], [4440214, 5743119], [3937670, 6103383], [8811288, 8091041], [8387214, 1530527], [5565653, 6735188], [3922027, 5963450], [-1, 8633246], [3912733, 8360014], [8393765, -1], [7950937, 5220677]]


From the above list, we can see which words are in the embeddings space and which are not. Suspicion confrimed that the emotional words "fears-confrimed" and "happy-for" is not in the embedding. Will need to find a replacement word.

Transpose the document-term matrix to term-document.

In [26]:
t_count_embeding = count_embeding.transpose()
t_count_embeding = t_count_embeding.astype(np.float64)
t_count_embeding

<11064651x24530 sparse matrix of type '<class 'numpy.float64'>'
	with 172907264 stored elements in Compressed Sparse Column format>

Get the negative and positive tokens, subtrack their vectors to generate a SemAxis and normalize the vector. Below is the semaxis for the polar words (Negative, Positive)

In [27]:
negative_vector = t_count_embeding[polar_word_index[0][0]]
positive_vector = t_count_embeding[polar_word_index[0][1]]
semaxis = positive_vector - negative_vector
semaxis_norm = normalize(semaxis, norm='l2', axis=1)
print(semaxis_norm[0, :])

  (0, 3)	0.0021937989419186853
  (0, 5)	0.0010968994709593426
  (0, 6)	0.0043875978838373705
  (0, 7)	0.0032906984128780277
  (0, 9)	0.0010968994709593426
  (0, 13)	0.0021937989419186853
  (0, 14)	0.0032906984128780277
  (0, 16)	0.0010968994709593426
  (0, 18)	-0.0010968994709593426
  (0, 19)	0.0032906984128780277
  (0, 20)	0.0010968994709593426
  (0, 24)	0.007678296296715398
  (0, 26)	0.0010968994709593426
  (0, 27)	0.0021937989419186853
  (0, 28)	0.0010968994709593426
  (0, 29)	-0.0021937989419186853
  (0, 35)	0.0032906984128780277
  (0, 36)	0.0021937989419186853
  (0, 37)	0.0021937989419186853
  (0, 40)	0.0021937989419186853
  (0, 41)	-0.0032906984128780277
  (0, 46)	0.0010968994709593426
  (0, 49)	0.0032906984128780277
  (0, 53)	0.0032906984128780277
  (0, 58)	0.005484497354796713
  :	:
  (0, 24463)	0.0032906984128780277
  (0, 24465)	-0.0010968994709593426
  (0, 24473)	0.005484497354796713
  (0, 24475)	0.008775195767674741
  (0, 24479)	-0.0010968994709593426
  (0, 24480)	-0.0010968

Project all token vectors onto the SemAxis vector.

In [28]:
def VectorProjection(a, b):
    numerator = a.dot(b)
    denominator = b.transpose().dot(b)
    frac = numerator/denominator
    return b.multiply(frac)

In [29]:
projected_points = t_count_embeding.copy()
print(projected_points.dtype)

for idex in range(t_count_embeding.shape[0]):
    projection = VectorProjection(t_count_embeding[idex], semaxis_norm.transpose())
    projected_points[idex] = projection.transpose()

float64


In [43]:
test_vector = projected_points[0].toarray()
print(test_vector[0, 3])

0.07578402767332579


In [48]:
from scipy.spatial import distance

scipy.spatial.distance.cdist(   XA= test_vector,
                                XB= semaxis.toarray(),
                                metric="euclidean")

ValueError: Unknown Distance Metric: euclidian

In [45]:
type(test_vector)

numpy.ndarray